In [1]:
import sys

from SALib.sample import saltelli
from tqdm import tqdm
import numpy as np
import pandas as pd
import pickle
import lzma
import os
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio


import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams.update({"font.size" : 15,
                     "figure.dpi" : 100, 
                     "grid.alpha" : 0.3, 
                     "axes.grid": True, 
                     "axes.axisbelow" : True,
                     "figure.figsize":(8,6),
                     "mathtext.fontset":"cm",
                     "xtick.labelsize": 14,
                     "ytick.labelsize": 14,
                     "axes.labelsize": 16, 
                     "legend.fontsize": 13.5})

plt.rc("text", usetex=False)
plt.rc("font", family="serif")

In [2]:
# problem definition
PROBLEM = {
    "num_vars" : 5,
    "names"    : ["theta",
                  "gain_right",
                  "saving_prop",
                  "prob_left",
                  "alpha"],
    "bounds"   : [[0.05, 0.95],
                  [1.70, 8.00],
                  [0.70, 0.80],
                  [0.30, 0.45],
                  [2.00, 12.0]]
}

# generate Saltelli samples
NUM_SAMPLES = 1024
X = saltelli.sample(PROBLEM, NUM_SAMPLES, calc_second_order=False)

<ipython-input-2-1647d18ac9f5>:18: DeprecationWarning: `salib.sample.saltelli` will be removed in SALib 1.5. Please use `salib.sample.sobol`
  X = saltelli.sample(PROBLEM, NUM_SAMPLES, calc_second_order=False)


In [3]:
with open("./community_regimes.pickle", "rb") as f:
    data = pickle.load(f)

In [4]:
regime_params = {0:[], 1:[], 2:[]}
for seed in data:
    for i,regime in enumerate(seed):
        regime_params[regime].append(X[i])
regime_params = {k: np.row_stack(v) for k,v in regime_params.items()}

In [5]:
upper_param_bounds = np.array([x[1] for x in PROBLEM["bounds"]])
lower_param_bounds = np.array([x[0] for x in PROBLEM["bounds"]])
bound_diff = upper_param_bounds - lower_param_bounds

categories = [r'$\theta$', r'$G_{upper}$', r'$\beta$', r'$\ell$', r'$\alpha$', r'$\theta$']
regimes = ["All Poor", "Some Rich", "All Rich"]

In [7]:
fig = make_subplots(rows=1, cols=3,
                    start_cell="top-left",
                    specs=[[{"type": "scatterpolar"}, {"type": "scatterpolar"}, {"type": "scatterpolar"}]],
                    subplot_titles=("All Poor", "Some Rich", "All Rich"))

for i in range(3):
    
    data=((np.mean(regime_params[i], axis=0)+np.std(regime_params[i], axis=0)) - lower_param_bounds)/bound_diff
    data_closed = np.concatenate([data, [data[0]]])
    fig.add_trace(go.Scatterpolar(
        r=data_closed,
        theta=categories,
        fill='toself',
        name="Average + 1 stdev",
        marker=dict(color='#1f77b4'),
        showlegend=i==0
    ), row=1, col=i+1)
        
    data=(np.mean(regime_params[i], axis=0) - lower_param_bounds) / bound_diff
    data_closed = np.concatenate([data, [data[0]]])
    fig.add_trace(go.Scatterpolar(
        r=data_closed,
        theta=categories,
        fill='toself',
        name="Average",
        marker=dict(color='#d62728'),
        showlegend=i==0,
    ), row=1, col=i+1)
    
    data=np.maximum(((np.mean(regime_params[i], axis=0)-np.std(regime_params[i], axis=0)) - lower_param_bounds)/bound_diff, 0)
    data_closed = np.concatenate([data, [data[0]]])
    fig.add_trace(go.Scatterpolar(
        r=data_closed,
        dr=0.2,
        theta=categories,
        fill='toself',
        name="Average - 1 stdev",
        marker=dict(color='forestgreen'),
        showlegend=i==0
    ), row=1, col=i+1)
    
fig.update_polars(
    radialaxis=dict(
        visible=True,
        range=[0, 1.05],
        dtick=0.2,
        tickangle=90
    ),
)
    
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.05,
    xanchor="left",
    x=0.41)
)
fig.update_layout(height=500, width=1000)
fig.write_image("radar_plot_community.pdf")
fig.show()